<a href="https://colab.research.google.com/github/nattaran/CC-7-Github/blob/master/VoicePipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


**Set Working Directory**

In [ ]:
import os

# Update this path to match your Google Drive folder
BASE_DIR = '/content/drive/MyDrive/health-tequity-case'
os.chdir(BASE_DIR)

# Verify
print(f"✅ Current directory: {os.getcwd()}")
print(f"📁 Files in directory: {os.listdir('.')}")

✅ Current directory: /content/drive/MyDrive/health-tequity-case
📁 Files in directory: ['Input_Audio_Files', 'Output_Audio_Files', 'Data', 'whisper_transcriptions_with_errors.csv', 'transcriptions_only.csv', 'error_rates_summary.csv']


**install Required Packages**














In [ ]:
# Install packages
!pip install -q openai-whisper
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q librosa soundfile
!pip install -q deep-translator
!pip install -q gtts
!pip install -q jiwer

print("✅ All packages installed successfully!")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
✅ All packages installed successfully!


**Import Libraries **

In [ ]:
import whisper
import os
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
from deep_translator import GoogleTranslator
from gtts import gTTS
import jiwer
from jiwer import wer, cer
from jiwer import process_words
import warnings
import torch
warnings.filterwarnings('ignore')


print("✅ All libraries imported successfully!")

# Update this path to match your Google Drive folder


✅ All libraries imported successfully!


**Accessing to the Spanish Audio Files **

In [ ]:
AUDIO_FOLDER = '/content/drive/MyDrive/health-tequity-case/Input_Audio_Files/'  # UPDATE THIS

# List all WAV files in the folder
audio_files = sorted([f for f in os.listdir(AUDIO_FOLDER) if f.endswith('.wav')])

print(f"📁 Found {len(audio_files)} WAV file(s):")
for filename in audio_files:
    print(f"  - {filename}")

📁 Found 4 WAV file(s):
  - q1_es.wav
  - q2_es.wav
  - q3_es.wav
  - q4_es.wav


In [ ]:
GROUND_TRUTH_SPANISH = {
    'q1_es.wav': '¿Cuáles son mis presiones sistólica y diastólica hoy?',
    'q2_es.wav': '¿Cuáles fueron los valores durante la última semana?',
    'q3_es.wav': '¿Cuál es la tendencia de los valores?',
    'q4_es.wav': '¿Cuáles son los rangos normales para una persona como yo?'
}

# Ground truth English translations
# UPDATE THESE with your actual ground truth translations
GROUND_TRUTH_ENGLISH = {
    'question1.wav': 'What are my systolic and diastolic blood pressures today?',
    'question2.wav': 'What were the values over the last week?',
    'question3.wav': 'What is the trend of the values?',
    'question4.wav': 'What are the normal ranges for a person like me?'
}

print("\n✅ Ground truth loaded for error rate calculations")


✅ Ground truth loaded for error rate calculations


In [ ]:
#  Load Whisper Model
def load_whisper_model(model_size="medium"):
    """
    Load Whisper model for ASR

    Args:
        model_size: 'tiny', 'base', 'small', 'medium', 'large'
                   'medium' recommended for Spanish (good accuracy/speed balance)

    Returns:
        Loaded Whisper model
    """
    print(f"🤖 Loading Whisper '{model_size}' model...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"   Device: {device}")

    model = whisper.load_model(model_size, device=device)
    print("✅ Model loaded successfully!")
    return model

# Load the model
model = load_whisper_model(model_size="medium")

🤖 Loading Whisper 'medium' model...
   Device: cpu
✅ Model loaded successfully!


# **Transcription Spanish Audio **

In [ ]:
def transcribe_spanish_audio(model, audio_path, task="transcribe"):
    """
    Transcribe Spanish audio file using Whisper

    Args:
        model: Loaded Whisper model
        audio_path: Path to WAV file
        task: 'transcribe' (Spanish->Spanish) or 'translate' (Spanish->English)

    Returns:
        Dictionary with transcription results
    """
    print(f"🎵 Processing: {audio_path}")

    result = model.transcribe(
        audio_path,
        language="spanish",  # Specify Spanish for better accuracy
        task=task,
        verbose=False
    )

    return result


# **Process All Audio Files**

In [ ]:
def process_audio_files(model, audio_folder, audio_files):
    """
    Process multiple Spanish audio files from Google Drive

    Args:
        model: Loaded Whisper model
        audio_folder: Path to folder containing audio files
        audio_files: List of audio filenames

    Returns:
        DataFrame with transcription results
    """
    results = []

    print("\n" + "="*60)
    print("🎯 TRANSCRIBING SPANISH AUDIO FILES")
    print("="*60)

    for i, audio_file in enumerate(audio_files, 1):
        audio_path = os.path.join(audio_folder, audio_file)
        print(f"\n[{i}/{len(audio_files)}] {audio_file}")

        if not os.path.exists(audio_path):
            print(f"⚠️  Warning: {audio_path} not found, skipping...")
            continue

        # Get Spanish transcription
        spanish_result = transcribe_spanish_audio(model, audio_path, task="transcribe")

        # Get English translation
        english_result = transcribe_spanish_audio(model, audio_path, task="translate")

        results.append({
            'question_number': i,
            'audio_file': audio_file,
            'spanish_transcription': spanish_result['text'].strip(),
            'english_translation': english_result['text'].strip(),
            'language_detected': spanish_result['language']
        })

        print(f"   🇪🇸 Spanish: {spanish_result['text'].strip()}")
        print(f"   🇬🇧 English: {english_result['text'].strip()}")

    return pd.DataFrame(results)

# Process all files from Google Drive
results_df = process_audio_files(model, AUDIO_FOLDER, audio_files)


🎯 TRANSCRIBING SPANISH AUDIO FILES

[1/4] q1_es.wav
🎵 Processing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q1_es.wav


100%|██████████| 324/324 [00:29<00:00, 11.15frames/s]


🎵 Processing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q1_es.wav


100%|██████████| 324/324 [00:27<00:00, 11.99frames/s]


   🇪🇸 Spanish: ¿Cuáles son mis valores de presión arterial hoy?
   🇬🇧 English: What are my blood pressure values today?

[2/4] q2_es.wav
🎵 Processing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q2_es.wav


...

   🇪🇸 Spanish: ¿Cuál es la tendencia de mis valores?
   🇬🇧 English: What is the trend of my values?

[4/4] q4_es.wav
🎵 Processing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q4_es.wav


100%|██████████| 398/398 [00:29<00:00, 13.44frames/s]


🎵 Processing: /content/drive/MyDrive/health-tequity-case/Input_Audio_Files/q4_es.wav


100%|██████████| 398/398 [00:28<00:00, 14.21frames/s]

   🇪🇸 Spanish: ¿Cuáles son los rangos normales para una persona como yo?
   🇬🇧 English: What are the normal ranges for a person like me?


** Error Rate Calculation Functions**

In [ ]:
def calculate_wer(reference, hypothesis):
    """
    Calculate Word Error Rate (WER)
    WER = (Substitutions + Deletions + Insertions) / Total Words in Reference

    Args:
        reference: Ground truth text
        hypothesis: Predicted/transcribed text

    Returns:
        WER score (0-1), detailed metrics
    """
    # Normalize text (lowercase, strip)
    ref = reference.lower().strip()
    hyp = hypothesis.lower().strip()

    # Calculate WER
    wer_score = wer(ref, hyp)

    # Get detailed measures
    output= process_words(ref, hyp)

    return {
        'wer': wer_score,
        'substitutions': output.substitutions,
        'deletions': output.deletions,
        'insertions': output.insertions,
        'hits': output.hits
    }

def calculate_cer(reference, hypothesis):
    """
    Calculate Character Error Rate (CER)
    CER = (Character Substitutions + Deletions + Insertions) / Total Characters

    Args:
        reference: Ground truth text
        hypothesis: Predicted/transcribed text

    Returns:
        CER score (0-1)
    """
    ref = reference.lower().strip()
    hyp = hypothesis.lower().strip()

    cer_score = cer(ref, hyp)
    return cer_score

def calculate_ser(reference, hypothesis):
    """
    Calculate Sentence Error Rate (SER)
    SER = 1 if sentences don't match exactly, 0 if they match

    Args:
        reference: Ground truth text
        hypothesis: Predicted/transcribed text

    Returns:
        SER score (0 or 1)
    """
    # Normalize for comparison
    ref = reference.lower().strip()
    hyp = hypothesis.lower().strip()

    # SER is 0 if exact match, 1 if not
    ser_score = 0 if ref == hyp else 1
    return ser_score

def calculate_all_error_rates(results_df, ground_truth_spanish):
    """
    Calculate all error rates for Spanish and English transcriptions

    Args:
        results_df: DataFrame with transcription results
        ground_truth_spanish: Dictionary of ground truth Spanish text
        ground_truth_english: Dictionary of ground truth English text

    Returns:
        DataFrame with error rates
    """
    error_rates = []

    for idx, row in results_df.iterrows():
        audio_file = row['audio_file']

        # Spanish error rates (ASR accuracy)
        if audio_file in ground_truth_spanish:
            spanish_gt = ground_truth_spanish[audio_file]
            spanish_hyp = row['spanish_transcription']

            spanish_wer_details = calculate_wer(spanish_gt, spanish_hyp)
            spanish_cer_score = calculate_cer(spanish_gt, spanish_hyp)
            spanish_ser_score = calculate_ser(spanish_gt, spanish_hyp)
        else:
            spanish_wer_details = {'wer': None, 'substitutions': None, 'deletions': None, 'insertions': None, 'hits': None}
            spanish_cer_score = None
            spanish_ser_score = None

        # English error rates (Translation accuracy)
        # if audio_file in ground_truth_english:
        #     english_gt = ground_truth_english[audio_file]
        #     english_hyp = row['english_translation']

        #     english_wer_details = calculate_wer(english_gt, english_hyp)
        #     english_cer_score = calculate_cer(english_gt, english_hyp)
        #     english_ser_score = calculate_ser(english_gt, english_hyp)
        # else:
        #     english_wer_details = {'wer': None, 'substitutions': None, 'deletions': None, 'insertions': None, 'hits': None}
        #     english_cer_score = None
        #     english_ser_score = None

        error_rates.append({
            'question_number': row['question_number'],
            'audio_file': audio_file,

            # Spanish metrics
            'spanish_wer': spanish_wer_details['wer'],
            'spanish_substitutions': spanish_wer_details['substitutions'],
            'spanish_deletions': spanish_wer_details['deletions'],
            'spanish_insertions': spanish_wer_details['insertions'],
            'spanish_cer': spanish_cer_score,
            'spanish_ser': spanish_ser_score,

            # English metrics
            # 'english_wer': english_wer_details['wer'],
            # 'english_substitutions': english_wer_details['substitutions'],
            # 'english_deletions': english_wer_details['deletions'],
            # 'english_insertions': english_wer_details['insertions'],
            # 'english_cer': english_cer_score,
            # 'english_ser': english_ser_score
        })

    return pd.DataFrame(error_rates)

# Calculate error rates
print("\n" + "="*60)
print("📊 CALCULATING ERROR RATES")
print("="*60)

error_rates_df = calculate_all_error_rates(results_df, GROUND_TRUTH_SPANISH)

# Merge with original results
full_results_df = results_df.merge(error_rates_df, on=['question_number', 'audio_file'])


📊 CALCULATING ERROR RATES


Display Results with Error Rates

In [ ]:
# ============================================
# CELL 7: Display Results with Error Rates
# ============================================
display_cols = ['question_number', 'audio_file', 'spanish_wer', 'spanish_cer',
                'spanish_ser']  #, 'english_wer', 'english_cer', 'english_ser']
print(full_results_df[display_cols].to_string(index=False))

# Calculate averages (skip None values)
print("\n" + "="*60)
print("📈 AVERAGE ERROR RATES")
print("="*60)

# Spanish metrics
spanish_wer_mean = full_results_df['spanish_wer'].dropna().mean()
spanish_cer_mean = full_results_df['spanish_cer'].dropna().mean()
spanish_ser_mean = full_results_df['spanish_ser'].dropna().mean()

print(f"Spanish ASR:")
if not pd.isna(spanish_wer_mean):
    print(f"  Average WER: {spanish_wer_mean:.4f} ({spanish_wer_mean*100:.2f}%)")
    print(f"  Average CER: {spanish_cer_mean:.4f} ({spanish_cer_mean*100:.2f}%)")
    print(f"  Average SER: {spanish_ser_mean:.4f} ({spanish_ser_mean*100:.2f}%)")
else:
    print(f"  ⚠️  No ground truth available for Spanish ASR")

# English metrics
# english_wer_mean = full_results_df['english_wer'].dropna().mean()
# english_cer_mean = full_results_df['english_cer'].dropna().mean()
# english_ser_mean = full_results_df['english_ser'].dropna().mean()

# print(f"\nEnglish Translation:")
# if not pd.isna(english_wer_mean):
#     print(f"  Average WER: {english_wer_mean:.4f} ({english_wer_mean*100:.2f}%)")
#     print(f"  Average CER: {english_cer_mean:.4f} ({english_cer_mean*100:.2f}%)")
#     print(f"  Average SER: {english_ser_mean:.4f} ({english_ser_mean*100:.2f}%)")
# else:
#     print(f"  ⚠️  No ground truth available for English translation")

# ============================================
# SAVE RESULTS
# ============================================
print("\n" + "="*60)
print("💾 SAVING RESULTS")
print("="*60)

# Save full results with all columns
output_file_full = "whisper_transcriptions_with_errors.csv"
full_results_df.to_csv(output_file_full, index=False)
print(f"✅ Full results saved to: {output_file_full}")

# Save transcriptions only (simplified)
transcriptions_only = full_results_df[['question_number', 'audio_file',
                                        'spanish_transcription', 'english_translation']].copy()
output_file_transcriptions = "transcriptions_only.csv"
transcriptions_only.to_csv(output_file_transcriptions, index=False)
print(f"✅ Transcriptions only saved to: {output_file_transcriptions}")

# Save error rates summary
error_summary = full_results_df[['question_number', 'audio_file',
                                  'spanish_wer', 'spanish_cer', 'spanish_ser']].copy()
output_file_errors = "error_rates_summary.csv"
error_summary.to_csv(output_file_errors, index=False)
print(f"✅ Error rates summary saved to: {output_file_errors}")

# Save to Google Drive (if mounted)
try:
    # Try to save to Drive if it's mounted
    drive_output_folder = '/content/drive/MyDrive/health_tequity_results/'
    os.makedirs(drive_output_folder, exist_ok=True)

    full_results_df.to_csv(os.path.join(drive_output_folder, output_file_full), index=False)
    transcriptions_only.to_csv(os.path.join(drive_output_folder, output_file_transcriptions), index=False)
    error_summary.to_csv(os.path.join(drive_output_folder, output_file_errors), index=False)

    print(f"\n✅ All files also saved to Google Drive: {drive_output_folder}")
except:
    print("\n⚠️  Google Drive not mounted - files saved locally only")

# Download files to your computer
print("\n📥 Download files to your computer:")
from google.colab import files
files.download(output_file_full)
files.download(output_file_transcriptions)
files.download(output_file_errors)

print("\n✅ All results saved successfully!")

 question_number audio_file  spanish_wer  spanish_cer  spanish_ser
               1  q1_es.wav     0.500000     0.452830            1
               2  q2_es.wav     0.125000     0.096154            1
               3  q3_es.wav     0.142857     0.054054            1
               4  q4_es.wav     0.000000     0.000000            0

📈 AVERAGE ERROR RATES
Spanish ASR:
  Average WER: 0.1920 (19.20%)
  Average CER: 0.1508 (15.08%)
  Average SER: 0.7500 (75.00%)

💾 SAVING RESULTS
✅ Full results saved to: whisper_transcriptions_with_errors.csv
✅ Transcriptions only saved to: transcriptions_only.csv
✅ Error rates summary saved to: error_rates_summary.csv

✅ All files also saved to Google Drive: /content/drive/MyDrive/health_tequity_results/

📥 Download files to your computer:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ All results saved successfully!


Save Results

Display Individual Results with Comparision

In [ ]:
print("\n" + "="*60)
print("📝 DETAILED TRANSCRIPTIONS WITH GROUND TRUTH COMPARISON")
print("="*60)

for idx, row in full_results_df.iterrows():
    print(f"\n{'='*60}")
    print(f"Question {row['question_number']}: {row['audio_file']}")
    print(f"{'='*60}")

    # Spanish comparison
    print(f"\n🇪🇸 SPANISH:")
    audio_file = row['audio_file']
    if audio_file in GROUND_TRUTH_SPANISH:
        print(f"  Ground Truth: {GROUND_TRUTH_SPANISH[audio_file]}")
        print(f"  Transcribed:  {row['spanish_transcription']}")

        # Display metrics if available
        if row['spanish_wer'] is not None:
            print(f"  WER: {row['spanish_wer']:.4f} | CER: {row['spanish_cer']:.4f} | SER: {row['spanish_ser']}")
            print(f"  Errors: Subs={row['spanish_substitutions']}, Dels={row['spanish_deletions']}, Ins={row['spanish_insertions']}")
        else:
            print(f"  ⚠️  No ground truth available for error calculation")
    else:
        print(f"  Transcribed:  {row['spanish_transcription']}")
        print(f"  ⚠️  No ground truth available")

    # # English comparison
    # print(f"\n🇬🇧 ENGLISH:")
    # if audio_file in GROUND_TRUTH_ENGLISH:
    #     print(f"  Ground Truth: {GROUND_TRUTH_ENGLISH[audio_file]}")
    #     print(f"  Translated:   {row['english_translation']}")

    #     # Display metrics if available
    #     if row['english_wer'] is not None:
    #         print(f"  WER: {row['english_wer']:.4f} | CER: {row['english_cer']:.4f} | SER: {row['english_ser']}")
    #         print(f"  Errors: Subs={row['english_substitutions']}, Dels={row['english_deletions']}, Ins={row['english_insertions']}")
    #     else:
    #         print(f"  ⚠️  No ground truth available for error calculation")
    # else:
    #     print(f"  Translated:   {row['english_translation']}")
    #     print(f"  ⚠️  No ground truth available")
    print()


📝 DETAILED TRANSCRIPTIONS WITH GROUND TRUTH COMPARISON

Question 1: q1_es.wav

🇪🇸 SPANISH:
  Ground Truth: ¿Cuáles son mis presiones sistólica y diastólica hoy?
  Transcribed:  ¿Cuáles son mis valores de presión arterial hoy?
  WER: 0.5000 | CER: 0.4528 | SER: 1
  Errors: Subs=4, Dels=0, Ins=0


Question 2: q2_es.wav

🇪🇸 SPANISH:
  Ground Truth: ¿Cuáles fueron los valores durante la última semana?
  Transcribed:  ¿Cuáles fueron los valores de la última semana?
  WER: 0.1250 | CER: 0.0962 | SER: 1
  Errors: Subs=1, Dels=0, Ins=0


Question 3: q3_es.wav

🇪🇸 SPANISH:
  Ground Truth: ¿Cuál es la tendencia de los valores?
  Transcribed:  ¿Cuál es la tendencia de mis valores?
  WER: 0.1429 | CER: 0.0541 | SER: 1
  Errors: Subs=1, Dels=0, Ins=0


Question 4: q4_es.wav

🇪🇸 SPANISH:
  Ground Truth: ¿Cuáles son los rangos normales para una persona como yo?
  Transcribed:  ¿Cuáles son los rangos normales para una persona como yo?
  WER: 0.0000 | CER: 0.0000 | SER: 0
  Errors: Subs=0, Dels=0, Ins

LLM **part**

In [ ]:
!pip -q install transformers accelerate bitsandbytes sentencepiece einops safetensors



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.9 MB/s eta 0:00:00


In [ ]:
# Optional: see what Colab gave you
!nvidia-smi || true

import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())

Fri Oct 17 18:25:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip -q install "openai==1.51.2" "httpx==0.27.2" "httpcore==1.0.5" pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which is incompatible.
google-genai 1.25.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


In [ ]:
from google.colab import userdata
api_key = userdata.get("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("Add OPENAI_API_KEY in the Secrets panel (left sidebar, key icon).")

In [ ]:
# Upload the CSV from google drive

from google.colab import drive
# drive.mount('/content/drive')
CSV_PATH = "/content/drive/MyDrive/health-tequity-case/Data/synthetic_bp_one_person.csv"

In [ ]:
# Load CSV as plain text
csv_text = open(CSV_PATH, "r", encoding="utf-8").read()
print("\n".join(csv_text.splitlines()[:5]))



date,age,sex,systolic_mmHg,diastolic_mmHg,regime,category
2025-09-17,68,Female,116,77,normal,normal
2025-09-18,68,Female,126,70,normal,elevated
2025-09-19,68,Female,123,81,normal,elevated
2025-09-20,68,Female,117,79,normal,normal


# **Define Templates + GPT call (JSON output)**

In [ ]:
import json, re
from openai import OpenAI

client = OpenAI(api_key=api_key)

TEMPLATES = {
    "today":         "Your systolic blood pressure was {sys} mm of Hg and your diastolic blood pressure was {dia} mm of Hg.",
    "last_week":     "Over the last week, your systolic blood pressure has averaged {sys_avg} mm of Hg and your diastolic blood pressure has averaged {dia_avg} mm of Hg.",
    "trend_month":   "The trend for the values over the last month has been {trend} average values of your systolic blood pressure and diastolic blood pressure.",
    "normal_ranges": "While each person’s normal range should be discussed with their physician, literature suggests that for a {sex} aged {age} years, systolic and diastolic blood pressure can typically be expected to be {sys_norm} mm Hg and {dia_norm} mm Hg respectively. This information was retrieved from {reference}."
}

SYSTEM = """You are a careful data analyst.
Do ALL analysis yourself using ONLY the CSV provided by the user.
Interpret columns: date, age, sex, systolic, diastolic.
- "Today" = most recent row by date.
- "Last week" = last 7 rows by date (including the most recent).
- "Trend over the last month" = last 30 rows; return one of: increasing / decreasing / stable.
- If a specific date is mentioned (e.g., 'on October 1'), return that date’s values if present.
Return STRICT JSON ONLY:
{
 "template": "today"|"last_week"|"trend_month"|"normal_ranges",
 "fields": {...},   # only the slots the chosen template needs (e.g., sys, dia, sys_avg, dia_avg, trend, age, sex, sys_norm, dia_norm, reference, date)
 "final_text": "one sentence exactly following the chosen template with mm of Hg units"
}
No extra prose. JSON only.
"""

def ask_gpt(question_en: str, csv_block: str) -> dict:
    user = f"CSV:\n{csv_block}\n\nQUESTION:\n{question_en}\n\nReturn JSON only."
    resp = client.chat.completions.create(
        model="gpt-4o",     # or "gpt-4o-mini" for lower cost
        temperature=0,
        messages=[
            {"role":"system","content":SYSTEM},
            {"role":"system","content":"Templates:\n" + json.dumps(TEMPLATES)},
            {"role":"user","content":user}
        ]
    ).choices[0].message.content

    # Strip possible code fences and parse only the JSON blob
    clean = re.sub(r"^```json|```$", "", resp.strip(), flags=re.M|re.I)
    start, end = clean.find("{"), clean.rfind("}")
    return json.loads(clean[start:end+1])



# **Run the 4 required question and save results**

In [ ]:
import pandas as pd, os

questions = [
    "What are my systolic and diastolic blood pressures today?",
    "What were the values over the last week?",
    "What is the trend of the values over the last month?",
    "What are the normal ranges for a person like me?",
    # Optional extension:
    "What were my blood pressure values on October 1?"
]

rows = []
for q in questions:
    obj = ask_gpt(q, csv_text)
    rows.append({
        "question": q,
        "template": obj.get("template"),
        "fields": json.dumps(obj.get("fields", {}), ensure_ascii=False),
        "answer_en": obj.get("final_text")
    })
    print(f"\nQ: {q}\nA: {obj.get('final_text')}")

os.makedirs("/content/data", exist_ok=True)
out_path = "/content/data/llm_answers_en_colab.csv"
pd.DataFrame(rows).to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"\n✅ Saved: {out_path}")



Q: What are my systolic and diastolic blood pressures today?
A: Your systolic blood pressure was 110 mm of Hg and your diastolic blood pressure was 76 mm of Hg.

Q: What were the values over the last week?
A: Over the last week, your systolic blood pressure has averaged 139 mm of Hg and your diastolic blood pressure has averaged 92 mm of Hg.

Q: What is the trend of the values over the last month?
A: The trend for the values over the last month has been increasing average values of your systolic blood pressure and diastolic blood pressure.

Q: What are the normal ranges for a person like me?
A: While each person’s normal range should be discussed with their physician, literature suggests that for a Female aged 68 years, systolic and diastolic blood pressure can typically be expected to be 120-129 mm Hg and 80-84 mm Hg respectively. This information was retrieved from European Society of Cardiology (ESC) guidelines.

Q: What were my blood pressure values on October 1?
A: Your systolic 